In [25]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 16: Chronal Classification ---</h2><p>As you see the Elves defend their hot chocolate successfully, you go back to falling through time. This is going to become a problem.</p>
<p>If you're ever going to return to your own time, you need to understand how this device on your wrist works. You have a little while before you reach your next destination, and with a bit of trial and error, you manage to pull up a programming manual on the device's tiny screen.</p>
<p>According to the manual, the device has four <a href="https://en.wikipedia.org/wiki/Hardware_register">registers</a> (numbered <code>0</code> through <code>3</code>) that can be manipulated by <a href="https://en.wikipedia.org/wiki/Instruction_set_architecture#Instructions">instructions</a> containing one of 16 opcodes. The registers start with the value <code>0</code>.</p>
<p>Every instruction consists of four values: an <em>opcode</em>, two <em>inputs</em> (named <code>A</code> and <code>B</code>), and an <em>output</em> (named <code>C</code>), in that order. The opcode specifies the behavior of the instruction and how the inputs are interpreted. The output, <code>C</code>, is always treated as a register.</p>
<p>In the opcode descriptions below, if something says "<em>value <code>A</code></em>", it means to take the number given as <code>A</code> <em>literally</em>. (This is also called an "immediate" value.) If something says "<em>register <code>A</code></em>", it means to use the number given as <code>A</code> to read from (or write to) the <em>register with that number</em>. So, if the opcode <code>addi</code> adds register <code>A</code> and value <code>B</code>, storing the result in register <code>C</code>, and the instruction <code>addi 0 7 3</code> is encountered, it would add <code>7</code> to the value contained by register <code>0</code> and store the sum in register <code>3</code>, never modifying registers <code>0</code>, <code>1</code>, or <code>2</code> in the process.</p>
<p>Many opcodes are similar except for how they interpret their arguments. The opcodes fall into seven general categories:</p>
<p>Addition:</p>
<ul>
<li><code>addr</code> (add register) stores into register <code>C</code> the result of adding register <code>A</code> and register <code>B</code>.</li>
<li><code>addi</code> (add immediate) stores into register <code>C</code> the result of adding register <code>A</code> and value <code>B</code>.</li>
</ul>
<p>Multiplication:</p>
<ul>
<li><code>mulr</code> (multiply register) stores into register <code>C</code> the result of multiplying register <code>A</code> and register <code>B</code>.</li>
<li><code>muli</code> (multiply immediate) stores into register <code>C</code> the result of multiplying register <code>A</code> and value <code>B</code>.</li>
</ul>
<p><a href="https://en.wikipedia.org/wiki/Bitwise_AND">Bitwise AND</a>:</p>
<ul>
<li><code>banr</code> (bitwise AND register) stores into register <code>C</code> the result of the bitwise AND of register <code>A</code> and register <code>B</code>.</li>
<li><code>bani</code> (bitwise AND immediate) stores into register <code>C</code> the result of the bitwise AND of register <code>A</code> and value <code>B</code>.</li>
</ul>
<p><a href="https://en.wikipedia.org/wiki/Bitwise_OR">Bitwise OR</a>:</p>
<ul>
<li><code>borr</code> (bitwise OR register) stores into register <code>C</code> the result of the bitwise OR of register <code>A</code> and register <code>B</code>.</li>
<li><code>bori</code> (bitwise OR immediate) stores into register <code>C</code> the result of the bitwise OR of register <code>A</code> and value <code>B</code>.</li>
</ul>
<p>Assignment:</p>
<ul>
<li><code>setr</code> (set register) copies the contents of register <code>A</code> into register <code>C</code>. (Input <code>B</code> is ignored.)</li>
<li><code>seti</code> (set immediate) stores value <code>A</code> into register <code>C</code>. (Input <code>B</code> is ignored.)</li>
</ul>
<p>Greater-than testing:</p>
<ul>
<li><code>gtir</code> (greater-than immediate/register) sets register <code>C</code> to <code>1</code> if value <code>A</code> is greater than register <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
<li><code>gtri</code> (greater-than register/immediate) sets register <code>C</code> to <code>1</code> if register <code>A</code> is greater than value <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
<li><code>gtrr</code> (greater-than register/register) sets register <code>C</code> to <code>1</code> if register <code>A</code> is greater than register <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
</ul>
<p>Equality testing:</p>
<ul>
<li><code>eqir</code> (equal immediate/register) sets register <code>C</code> to <code>1</code> if value <code>A</code> is equal to register <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
<li><code>eqri</code> (equal register/immediate) sets register <code>C</code> to <code>1</code> if register <code>A</code> is equal to value <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
<li><code>eqrr</code> (equal register/register) sets register <code>C</code> to <code>1</code> if register <code>A</code> is equal to register <code>B</code>. Otherwise, register <code>C</code> is set to <code>0</code>.</li>
</ul>
<p>Unfortunately, while the manual gives the <em>name</em> of each opcode, it doesn't seem to indicate the <em>number</em>. However, you can monitor the CPU to see the contents of the registers before and after instructions are executed to try to work them out.  Each opcode has a number from <code>0</code> through <code>15</code>, but the manual doesn't say which is which. For example, suppose you capture the following sample:</p>
<pre><code>Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]
</code></pre>
<p>This sample shows the effect of the instruction <code>9 2 1 2</code> on the registers. Before the instruction is executed, register <code>0</code> has value <code>3</code>, register <code>1</code> has value <code>2</code>, and registers <code>2</code> and <code>3</code> have value <code>1</code>. After the instruction is executed, register <code>2</code>'s value becomes <code>2</code>.</p>
<p>The instruction itself, <code>9 2 1 2</code>, means that opcode <code>9</code> was executed with <code>A=2</code>, <code>B=1</code>, and <code>C=2</code>. Opcode <code>9</code> could be any of the 16 opcodes listed above, but only three of them behave in a way that would cause the result shown in the sample:</p>
<ul>
<li>Opcode <code>9</code> could be <code>mulr</code>: register <code>2</code> (which has a value of <code>1</code>) times register <code>1</code> (which has a value of <code>2</code>) produces <code>2</code>, which matches the value stored in the output register, register <code>2</code>.</li>
<li>Opcode <code>9</code> could be <code>addi</code>: register <code>2</code> (which has a value of <code>1</code>) plus value <code>1</code> produces <code>2</code>, which matches the value stored in the output register, register <code>2</code>.</li>
<li>Opcode <code>9</code> could be <code>seti</code>: value <code>2</code> matches the value stored in the output register, register <code>2</code>; the number given for <code>B</code> is irrelevant.</li>
</ul>
<p>None of the other opcodes produce the result captured in the sample. Because of this, the sample above <em>behaves like three opcodes</em>.</p>
<p>You collect many of these samples (the first section of your puzzle input). The manual also includes a small test program (the second section of your puzzle input) - you can <em>ignore it for now</em>.</p>
<p>Ignoring the opcode numbers, <em>how many samples in your puzzle input behave like three or more opcodes?</em></p>
</article>


In [26]:
example = """
Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]
"""


def parse(s: str) -> list[list[list[int]]]:
    cpts = []
    for capture in re.split(r"\n\s*\n", s.strip()):
        cp = capture.splitlines()
        if len(cp) == 3:
            cpts.append([[int(i) for i in re.findall(r"-?\d+", p)] for p in cp])

    return cpts


def count_valid_opcodes(before: list[int], opcodes: list[int], after: list[int]):
    _, A, B, C = opcodes
    count = 0
    # Addition:
    # addr (add register) stores into register C the result of
    # adding register A and register B.
    result = before[:]
    result[C] = result[A] + result[B]
    if result == after:
        count += 1
    # addi (add immediate) stores into register C the result of
    # adding register A and value B.
    result = before[:]
    result[C] = result[A] + B
    if result == after:
        count += 1

    # Multiplication:
    # mulr (multiply register) stores into register C the result
    # of multiplying register A and register B.
    result = before[:]
    result[C] = result[A] * result[B]
    if result == after:
        count += 1
    # muli (multiply immediate) stores into register C the result
    # of multiplying register A and value B.
    result = before[:]
    result[C] = result[A] * B
    if result == after:
        count += 1

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result
    # of the bitwise AND of register A and register B.
    result = before[:]
    result[C] = result[A] & result[B]
    if result == after:
        count += 1
    # bani (bitwise AND immediate) stores into register C the result
    # of the bitwise AND of register A and value B.
    result = before[:]
    result[C] = result[A] & B
    if result == after:
        count += 1

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result
    # of the bitwise OR of register A and register B.
    result = before[:]
    result[C] = result[A] | result[B]
    if result == after:
        count += 1
    # bori (bitwise OR immediate) stores into register C the result
    # of the bitwise OR of register A and value B.
    result = before[:]
    result[C] = result[A] | B
    if result == after:
        count += 1

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    result = before[:]
    result[C] = result[A]
    if result == after:
        count += 1

    # seti (set immediate) stores value A into register C. (Input B is ignored.)
    result = before[:]
    result[C] = A
    if result == after:
        count += 1

    # Greater-than testing:
    # gtir (greater-than immediate/register)
    # sets register C to 1 if value A is greater than register B.
    # Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if A > result[B] else 0
    if result == after:
        count += 1

    # gtri (greater-than register/immediate) sets register C to 1
    # if register A is greater than value B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] > B else 0
    if result == after:
        count += 1
    # gtrr (greater-than register/register) sets register C to 1
    # if register A is greater than register B. Otherwise, register C
    # is set to 0.
    result = before[:]
    result[C] = 1 if result[A] > result[B] else 0
    if result == after:
        count += 1

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1
    # if value A is equal to register B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if A == result[B] else 0
    if result == after:
        count += 1
    # eqri (equal register/immediate) sets register C to 1
    # if register A is equal to value B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] == B else 0
    if result == after:
        count += 1
    # eqrr (equal register/register) sets register C to 1
    # if register A is equal to register B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] == result[B] else 0
    if result == after:
        count += 1

    return count


def count_more_then_n_valid_opcodes(s: str, n: int = 3) -> int:
    captures = parse(s)
    count = 0
    for before, opcodes, after in captures:
        if count_valid_opcodes(before, opcodes, after) >= n:
            count += 1
    return count


print(f"Example: {count_more_then_n_valid_opcodes(example)} should be 1")

Example: 1 should be 1


In [27]:
with open("../input/day16.txt") as f:
    puzzle = f.read()

count_more_then_n_valid_opcodes(puzzle)
print(f"Part I: {count_more_then_n_valid_opcodes(puzzle)}")

Part I: 642


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>642</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Using the samples you collected, <span title="This is one of my favorite puzzles.">work out the number of each opcode</span> and execute the test program (the second section of your puzzle input).</p>
<p><em>What value is contained in register <code>0</code> after executing the test program?</em></p>
</article>

</main>


In [28]:
from collections import Counter
from unittest import result


example = """
Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]
"""


def parse_II(s: str) -> list[list[list[int]]]:
    progam = []
    for capture in re.split(r"\n\s*\n", s.strip()):
        lines = capture.splitlines()
        if len(lines) > 3:
            progam.extend(
                [[int(i) for i in re.findall(r"\d+", inst)] for inst in lines]
            )
    return progam


def validate(
    before: list[int],
    opcodes: list[int],
    after: list[int],
    correct_opcodes: defaultdict[int, list[str]],
) -> None:
    opcode, A, B, C = opcodes
    # Addition:
    # addr (add register) stores into register C the result of
    # adding register A and register B.
    result = before[:]
    result[C] = result[A] + result[B]
    if result == after:
        correct_opcodes[opcode].append("addr")
    # addi (add immediate) stores into register C the result of
    # adding register A and value B.
    result = before[:]
    result[C] = result[A] + B
    if result == after:
        correct_opcodes[opcode].append("addi")

    # Multiplication:
    # mulr (multiply register) stores into register C the result
    # of multiplying register A and register B.
    result = before[:]
    result[C] = result[A] * result[B]
    if result == after:
        correct_opcodes[opcode].append("mulr")
    # muli (multiply immediate) stores into register C the result
    # of multiplying register A and value B.
    result = before[:]
    result[C] = result[A] * B
    if result == after:
        correct_opcodes[opcode].append("muli")

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result
    # of the bitwise AND of register A and register B.
    result = before[:]
    result[C] = result[A] & result[B]
    if result == after:
        correct_opcodes[opcode].append("banr")
    # bani (bitwise AND immediate) stores into register C the result
    # of the bitwise AND of register A and value B.
    result = before[:]
    result[C] = result[A] & B
    if result == after:
        correct_opcodes[opcode].append("bani")

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result
    # of the bitwise OR of register A and register B.
    result = before[:]
    result[C] = result[A] | result[B]
    if result == after:
        correct_opcodes[opcode].append("borr")
    # bori (bitwise OR immediate) stores into register C the result
    # of the bitwise OR of register A and value B.
    result = before[:]
    result[C] = result[A] | B
    if result == after:
        correct_opcodes[opcode].append("bori")

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    result = before[:]
    result[C] = result[A]
    if result == after:
        correct_opcodes[opcode].append("setr")

    # seti (set immediate) stores value A into register C. (Input B is ignored.)
    result = before[:]
    result[C] = A
    if result == after:
        correct_opcodes[opcode].append("seti")

    # Greater-than testing:
    # gtir (greater-than immediate/register)
    # sets register C to 1 if value A is greater than register B.
    # Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if A > result[B] else 0
    if result == after:
        correct_opcodes[opcode].append("gtir")

    # gtri (greater-than register/immediate) sets register C to 1
    # if register A is greater than value B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] > B else 0
    if result == after:
        correct_opcodes[opcode].append("gtri")
    # gtrr (greater-than register/register) sets register C to 1
    # if register A is greater than register B. Otherwise, register C
    # is set to 0.
    result = before[:]
    result[C] = 1 if result[A] > result[B] else 0
    if result == after:
        correct_opcodes[opcode].append("gtrr")

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1
    # if value A is equal to register B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if A == result[B] else 0
    if result == after:
        correct_opcodes[opcode].append("eqir")
    # eqri (equal register/immediate) sets register C to 1
    # if register A is equal to value B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] == B else 0
    if result == after:
        correct_opcodes[opcode].append("eqri")
    # eqrr (equal register/register) sets register C to 1
    # if register A is equal to register B. Otherwise, register C is set to 0.
    result = before[:]
    result[C] = 1 if result[A] == result[B] else 0
    if result == after:
        correct_opcodes[opcode].append("eqrr")


def get_opcode_counts(s: str, n: int = 3) -> dict[int, Counter]:
    captures = parse(s)
    correct_opcodes = defaultdict(list[int])
    for before, opcodes, after in captures:
        validate(before, opcodes, after, correct_opcodes)
    return dict((k, Counter(v)) for k, v in correct_opcodes.items())


def get_opcodes(correct_opcodes: dict[int, Counter]) -> list[str]:
    codes = [None] * 16
    while any(len(c) > 1 for c in correct_opcodes.values()):
        for opc, count in list(correct_opcodes.items()):
            if len(count) == 1:
                value, _ = count.popitem()
                del correct_opcodes[opc]
                codes[opc] = value
                for opc1 in correct_opcodes:
                    if value in correct_opcodes[opc1]:
                        del correct_opcodes[opc1][value]
                        if not correct_opcodes[opc1]:
                            del correct_opcodes[opc1]

    return codes


def process(codes: list[int], instruction: list[int], registers: list[int]) -> None:
    code, A, B, C = instruction
    inst = codes[code]
    # Addition:
    # addr (add register) stores into register C the result of
    # adding register A and register B.
    if inst == "addr":
        registers[C] = registers[A] + registers[B]
    # addi (add immediate) stores into register C the result of
    # adding register A and value B.
    elif inst == "addi":
        registers[C] = registers[A] + B

    # Multiplication:
    # mulr (multiply register) stores into register C the result
    # of multiplying register A and register B.
    elif inst == "mulr":
        registers[C] = registers[A] * registers[B]
    # muli (multiply immediate) stores into register C the result
    # of multiplying register A and value B.
    elif inst == "muli":
        registers[C] = registers[A] * B

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result
    # of the bitwise AND of register A and register B.
    elif inst == "banr":
        registers[C] = registers[A] & registers[B]
    # bani (bitwise AND immediate) stores into register C the result
    # of the bitwise AND of register A and value B.
    elif inst == "bani":
        registers[C] = registers[A] & B

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result
    # of the bitwise OR of register A and register B.
    elif inst == "borr":
        registers[C] = registers[A] | registers[B]
    # bori (bitwise OR immediate) stores into register C the result
    # of the bitwise OR of register A and value B.
    elif inst == "bori":
        registers[C] = registers[A] | B

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    elif inst == "setr":
        registers[C] = registers[A]
    # seti (set immediate) stores value A into register C. (Input B is ignored.)
    elif inst == "seti":
        registers[C] = A

    # Greater-than testing:
    # gtir (greater-than immediate/register)
    # sets register C to 1 if value A is greater than register B.
    # Otherwise, register C is set to 0.
    elif inst == "gtir":
        registers[C] = 1 if A > registers[B] else 0

    # gtri (greater-than register/immediate) sets register C to 1
    # if register A is greater than value B. Otherwise, register C is set to 0.
    elif inst == "gtri":
        registers[C] = 1 if registers[A] > B else 0
    # gtrr (greater-than register/register) sets register C to 1
    # if register A is greater than register B. Otherwise, register C
    # is set to 0.
    elif inst == "gtrr":
        registers[C] = 1 if registers[A] > registers[B] else 0

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1
    # if value A is equal to register B. Otherwise, register C is set to 0.
    elif inst == "eqir":
        registers[C] = 1 if A == registers[B] else 0
    # eqri (equal register/immediate) sets register C to 1
    # if register A is equal to value B. Otherwise, register C is set to 0.
    elif inst == "eqri":
        registers[C] = 1 if registers[A] == B else 0
    # eqrr (equal register/register) sets register C to 1
    # if register A is equal to register B. Otherwise, register C is set to 0.
    elif inst == "eqrr":
        registers[C] = 1 if registers[A] == registers[B] else 0


def run(s: str) -> int:
    codes = get_opcodes(get_opcode_counts(s))
    program = parse_II(s)
    registers = [0] * 4

    for instruction in program:
        process(codes, instruction, registers)
    return registers[0]


print(f"Part II: {run(puzzle)}")

Part II: 481


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>481</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
